In [2]:
%pip install tensorflow

  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached protobuf-4.22.3-cp39-cp39-win_amd64.whl (420 kB)
  Using cached numpy-1.23.5-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached numpy-1.22.4-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached grpcio-1.54.0-cp39-cp39-win_amd64.whl (4.1 MB)
  Using cached google_auth-2.17.3-py2.py3-none-any.whl (178 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.33.0
    Uninstalling google-auth-1.33.0:
Note: you may need to restart the kernel to use updated packages.      Successfully uninstalled google-auth-1.33.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.4 which is incompatible.
google-cloud-storage 1.31.0 requires google-auth<2.0dev,>=1.11.0, but you have google-auth 2.17.3 which is incompatible.
google-cloud-core 1.7.1 requires google-auth<2.0dev,>=1.24.0, but you have google-auth 2.17.3 which is incompatible.
google-api-core 1.25.1 requires google-auth<2.0dev,>=1.21.1, but you have google-auth 2.17.3 which is incompatible.


In [3]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install serial

Note: you may need to restart the kernel to use updated packages.


In [2]:
from keras.models import Model
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16, decode_predictions
import matplotlib.pyplot as plt
import numpy as np
import cv2
import serial
import time
import os
from pylab import imread
import matplotlib.pyplot as plt
import urllib.request
from PIL import Image

In [3]:
def vib():
    print("Watch Out")
    board.write(b'H')
    time.sleep(1)
    board.write(b'L')

In [5]:
model = VGG16(weights = 'imagenet', include_top=True)
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

board = serial.Serial('COM4') #실행 전 포트번호 체크
time.sleep(2)

url = 'http://172.30.1.4/cam-hi.jpg' #실행 전 ESP-CAM 주소 확인
winName = 'ESP32 CAMERA'

cv2.namedWindow(winName, cv2.WINDOW_AUTOSIZE)

while(1):
    try:
        imgResponse = urllib.request.urlopen(url)
        imgNp = np.array(bytearray(imgResponse.read()), dtype=np.uint8)
        img = cv2.imdecode(imgNp, -1)
        
        image = np.array(Image.open(urllib.request.urlopen(url)))
        
        im = cv2.resize(image, (224,224))
        im = np.expand_dims(im, axis = 0)

        out = model.predict(im)
        decode_predic = decode_predictions(out, top=3)[0]

        mostsim, secsim, thdsim = decode_predic
        print('1. %s (%.2f%%)'% (mostsim[1], mostsim[2]*100))
        print('2. %s (%.2f%%)'% (secsim[1], secsim[2]*100))
        
        cv2.imshow(winName,img)
        
        if mostsim[1] == "ballpoint":
            vib()
    
    except:
        pass

    #ESC key 입력
    tecla = cv2.waitKey(5) & 0xFF
    if tecla == 27:
        break

#while문 벗어나면 실행
cv2.destroyAllWindows()
board.close()

AttributeError: module 'serial' has no attribute 'Serial'